In [1]:
import pandas as pd
import numpy as np

import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

komentar = pd.read_csv("comments_youtube.csv")
#menampilkan data komentar kkb
komentar.head(5)

#------Case FOlding------
komentar['Comments_youtube'] = komentar['Comments_youtube'].str.lower()
print('Case Folding Result : \n')
print(komentar['Comments_youtube'].head(5))

Case Folding Result : 

0                              nah dapet juga akhirnya
1    senang lihat klu perusuh teroris tertangkap\ns...
2    jangan bilang kalau di lamban  itu melanggar h...
3    terbangkan dan curah dari ketinggian 50.000 ka...
4    bravo tni,polri....tuntas habis pangacau sudah...
Name: Comments_youtube, dtype: object


In [2]:
# ------ Tokenizing ---------
def remove_char_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+?)|(\w+:\/\/\S+)","", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

komentar['Comments_youtube'] = komentar['Comments_youtube'].apply(remove_char_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

komentar['Comments_youtube'] = komentar['Comments_youtube'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

komentar['Comments_youtube'] = komentar['Comments_youtube'].apply(remove_punctuation)

#remove special_char
def remove_special_char(text):
    text = text = text.replace("\n"," ") 
    text = re.sub(r"\,", "", text)
    text = re.sub(r"\.", "", text)
    text = re.sub(r"\/", "", text)
    return  re.sub(r"\?", "", text)

komentar['Comments_youtube'] = komentar['Comments_youtube'].apply(remove_special_char)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

komentar['Comments_youtube'] = komentar['Comments_youtube'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

komentar['Comments_youtube'] = komentar['Comments_youtube'].apply(remove_whitespace_multiple)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

komentar['Comments_tokens'] =  komentar['Comments_youtube'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
komentar['Comments_tokens'].head(1)


Tokenizing Result : 



0    [nah, dapet, juga, akhirnya]
Name: Comments_tokens, dtype: object

In [3]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

komentar['comments_tokens_fdist'] = komentar['Comments_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
komentar['comments_tokens_fdist'].head().apply(lambda x : x.most_common())

Frequency Tokens : 



0     [(nah, 1), (dapet, 1), (juga, 1), (akhirnya, 1)]
1    [(tertangkap, 2), (senang, 1), (lihat, 1), (kl...
2    [(yang, 2), (jangan, 1), (bilang, 1), (kalau, ...
3    [(bravo, 2), (terbangkan, 1), (dan, 1), (curah...
4    [(bravo, 1), (tnipolrituntas, 1), (habis, 1), ...
Name: comments_tokens_fdist, dtype: object

In [4]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh','nya', 'n', 't', 'tp'
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah','tp','cocokkkkkk','robbb','ampung',
                        'hastag', 'vanuatu','min','mu','ku','up','job','moro','th',
                        'tnipolrituntas','pa'])

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

komentar['Comments_stopword'] = komentar['Comments_tokens'].apply(stopwords_removal) 


print(komentar['Comments_stopword'].head(5))

0                                              [dapet]
1    [senang, lihat, klu, perusuh, teroris, tertang...
2    [lamban, melanggar, ham, lihat, lakukan, suste...
3    [terbangkan, curah, ketinggian, kaki, bravo, t...
4    [bravo, habis, pangacau, bnyak, nywa, tni, mel...
Name: Comments_stopword, dtype: object


In [5]:
normalizad_word = pd.read_excel("Normalisasi_komentar.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

komentar['Comments_normalized'] = komentar['Comments_stopword'].apply(normalized_term)

komentar['Comments_normalized'].head(60)

0                                               [dapat]
1     [senang, lihat, kalau, perusuh, teroris, terta...
2     [lamban, melanggar, ham, lihat, lakukan, suste...
3     [terbangkan, curah, ketinggian, kaki, selamat,...
4     [selamat, habis, pangacau, banyak, nyawa, tni,...
5     [introgasi, setelahnya, tembak, selamat, tni, ...
6                              [allah, melundungi, tni]
7                         [jadikan, umpan, teman, saja]
8     [semangat, kapolda, habisi, teroris, meresahka...
9     [semoga, anak, buah, kakak, kawan, ciptakan, k...
10             [salut, anggota, tnipolri, tahan, sabar]
11               [kandangin, sama, singa, lapar, buaya]
12                                             [mantap]
13    [langsung, eksekusi, suruh, baris, lapangan, t...
14    [tolong, berita, ribuan, cpmi, mangkrak, kepas...
15    [tembak, tembak, warga, tidak, bersalah, nanti...
16                                             [sangat]
17                                      [alhamdu

In [6]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in komentar['Comments_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

komentar['Comments_tokens_stemmed'] = komentar['Comments_normalized'].swifter.apply(get_stemmed_term)
print(komentar['Comments_tokens_stemmed'])

873
------------------------
dapat : dapat
senang : senang
lihat : lihat
kalau : kalau
perusuh : rusuh
teroris : teroris
tertangkap : tangkap
smg : smg
rekannya : rekan
cepat : cepat
selamat : selamat
tnipolri : tnipolri
semoga : moga
lindungan : lindung
allah : allah
swt : swt
aamiin : aamiin
yra : yra
lamban : lamban
melanggar : langgar
ham : ham
lakukan : laku
suster : suster
surter : surter
bersalah : salah
terbangkan : terbang
curah : curah
ketinggian : tinggi
kaki : kaki
kapolda : kapolda
habis : habis
pangacau : pangacau
banyak : banyak
nyawa : nyawa
tni : tni
melayang : layang
pembalasan : balas
introgasi : introgasi
setelahnya : telah
tembak : tembak
polri : polri
melundungi : melundungi
jadikan : jadi
umpan : umpan
teman : teman
saja : saja
semangat : semangat
habisi : habis
meresahkan : resah
masyarakat : masyarakat
anak : anak
buah : buah
kakak : kakak
kawan : kawan
ciptakan : cipta
kedamaian : damai
papua : papua
nkri : nkri
harga : harga
mati : mati
salut : salut
anggota 

persbunyian : persbunyian
mantep : mantep
bgt : bgt
pas : pas
live : live
tonton : tonton
siaran : siar
dooorr : dooorr
kepalanyaa : kepalanyaa
jinakin : jinakin
ranjau : ranjau
apapun : apa
mreka : mreka
manfaatkan : manfaat
bertanggung : tanggung
gua : gua
galfok : galfok
inggusnya : inggusnya
kamar : kamar
kemudia : kemudia
karbit : karbit
masak : masak
door : door
ndan : ndan
dipihak : pihak
uda : uda
lucuti : lucut
senjatanya : senjata
kama : kama
tunjukkan : tunjuk
sembunyi : sembunyi
sampe : sampe
ketemu : ketemu
geram : geram
selang : selang
lagend : lagend
ktmu : ktmu
main : main
xanana : xanana
gusmao : gusmao
cerita : cerita
bertugas : tugas
kayak : kayak
gini : gin
lgsg : lgsg
dimatiin : dimatiin
lahjgn : lahjgn
lagibikin : lagibikin
sempit : sempit
lapas : lapas
doang : doang
mending : mending
hancur : hancur
beratnyadgn : beratnyadgn
perbuatan : buat
terhadapnakes : terhadapnakes
wanita : wanita
susah : susah
payah : payah
kali : kali
seharidari : seharidari
hutan : hutan

Pandas Apply:   0%|          | 0/220 [00:00<?, ?it/s]

0                                                [dapat]
1      [senang, lihat, kalau, rusuh, teroris, tangkap...
2      [lamban, langgar, ham, lihat, laku, suster, su...
3      [terbang, curah, tinggi, kaki, selamat, tnipol...
4      [selamat, habis, pangacau, banyak, nyawa, tni,...
                             ...                        
215                  [kafir, dor, hallal, toyib, pahala]
216                        [cincang, kasih, makan, babi]
217    [dpo, lepas, kirim, terus, nusakambangan, beta...
218    [moga, tangkap, sadar, salah, tangkap, moga, a...
219    [iket, saja, tengah, kasih, bom, cemplungin, l...
Name: Comments_tokens_stemmed, Length: 220, dtype: object


In [7]:
komentar.to_csv("comment_text_Preprocessing.csv")